##Instalando dependencias 


In [60]:
 !pip install 'fsspec>=0.3.3'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Imports


In [61]:
import dask.dataframe as dd
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor


##Importando dados

In [62]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Verificando dataset

In [63]:
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_new_df.csv')

In [64]:
df = df.compute()

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111237 entries, 0 to 111236
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               111237 non-null  float64
 1   order_item_id          111237 non-null  float64
 2   product_id             111237 non-null  float64
 3   seller_id              111237 non-null  float64
 4   price                  111237 non-null  float64
 5   freight_value          111237 non-null  float64
 6   payment_type           111237 non-null  float64
 7   payment_installments   111237 non-null  float64
 8   payment_value          111237 non-null  float64
 9   customer_unique_id     111237 non-null  float64
 10  product_category_name  111237 non-null  float64
 11  product_weight_g       111237 non-null  float64
 12  product_length_cm      111237 non-null  float64
 13  product_height_cm      111237 non-null  float64
 14  product_width_cm       111237 non-nu

In [66]:
df.head(10)

,order_id,order_item_id,product_id,seller_id,price,freight_value,payment_type,payment_installments,payment_value,customer_unique_id,...,seller_lat,seller_lng,days_to_delivery,early_delivery,day_shipping,month_shipping,year_shipping,day_order_approved,month_order_approved,year_order_approved
0,0.0,1.0,8216.0,812.0,58.9,13.29,1.0,2.0,72.19,48076.0,...,-22.497188,-44.127324,6.0,10.0,19.0,9.0,2017.0,13.0,9.0,2017.0
1,7014.0,1.0,8216.0,812.0,55.9,17.96,0.0,1.0,73.86,5649.0,...,-22.497188,-44.127324,6.0,21.0,5.0,7.0,2017.0,29.0,6.0,2017.0
2,30559.0,1.0,8216.0,812.0,64.9,18.33,1.0,2.0,83.23,18589.0,...,-22.497188,-44.127324,5.0,15.0,23.0,5.0,2018.0,18.0,5.0,2018.0
3,40954.0,1.0,8216.0,812.0,58.9,16.17,1.0,3.0,75.07,82561.0,...,-22.497188,-44.127324,1.0,23.0,7.0,8.0,2017.0,1.0,8.0,2017.0
4,45875.0,1.0,8216.0,812.0,58.9,13.29,1.0,4.0,72.19,4247.0,...,-22.497188,-44.127324,1.0,21.0,16.0,8.0,2017.0,10.0,8.0,2017.0
5,58794.0,1.0,8216.0,812.0,55.9,26.93,1.0,1.0,82.83,18290.0,...,-22.497188,-44.127324,5.0,22.0,3.0,8.0,2017.0,28.0,7.0,2017.0
6,69027.0,1.0,8216.0,812.0,64.9,38.50,0.0,1.0,103.40,17734.0,...,-22.497188,-44.127324,0.0,23.0,26.0,3.0,2018.0,20.0,3.0,2018.0
7,74190.0,2.0,8216.0,812.0,58.9,18.12,0.0,1.0,153.75,47060.0,...,-22.497188,-44.127324,2.0,27.0,14.0,8.0,2017.0,8.0,8.0,2017.0
8,74190.0,1.0,8359.0,812.0,58.9,17.83,0.0,1.0,153.75,47060.0,...,-22.497188,-44.127324,2.0,27.0,14.0,8.0,2017.0,8.0,8.0,2017.0
9,82148.0,1.0,8216.0,812.0,55.9,35.71,3.0,1.0,20.00,5361.0,...,-22.497188,-44.127324,2.0,29.0,9.0,6.0,2017.0,5.0,6.0,2017.0


##Preparação dos dados


In [67]:
df.isnull().sum()

order_id                 0
order_item_id            0
product_id               0
seller_id                0
price                    0
freight_value            0
payment_type             0
payment_installments     0
payment_value            0
customer_unique_id       0
product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
costumer_lat             0
costumer_lng             0
seller_lat               0
seller_lng               0
days_to_delivery         0
early_delivery           0
day_shipping             0
month_shipping           0
year_shipping            0
day_order_approved       0
month_order_approved     0
year_order_approved      0
dtype: int64

In [68]:
#transformando em float
df = df.astype(float)

In [69]:
#removendo dados NAN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [70]:
colunas = ['Modelo','MAE','MAPE'] 
Metrics = pd.DataFrame(columns = colunas)

In [71]:
Metrics

,Modelo,MAE,MAPE


In [72]:
##separando as variaveis dependente e independente em dois data frames
X = df.drop(['days_to_delivery','early_delivery'], 1)
y = df['days_to_delivery']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [73]:
# Splitting the dataset into 80% training data and 20% testing data.
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.20, random_state=0)

In [74]:
from sklearn.model_selection import KFold

kf = KFold(5, shuffle=True, random_state=150)

#Modelos


##Linear Regression

In [75]:
model = LinearRegression()

my_pipeline = Pipeline(steps=[('m', model)])

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_train, Y_train,
                        cv=kf,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'LinearRegression',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)

In [76]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15


##Decision Tree


In [77]:
model = DecisionTreeRegressor(random_state=150)

my_pipeline = Pipeline(steps=[('m', model)])

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_test, Y_test,
                        cv=kf,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'Decision Tree',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)

In [78]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15
1,Decision Tree,2.250492,1.362130e+15


##Random forest trees

In [79]:
model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=0)

my_pipeline = Pipeline(steps=[('m', model)])

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_test, Y_test,
                        cv=kf,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'Random Forest',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)

In [80]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15
1,Decision Tree,2.250492,1.362130e+15
2,Random Forest,1.811036,1.598093e+15


##XGBoost

In [81]:
model = xgb.XGBRegressor(silent = True)

my_pipeline = Pipeline(steps=[('m', model)])

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_test, Y_test, #test
                        cv=kf,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'XGB',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)

In [82]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15
1,Decision Tree,2.250492,1.362130e+15
2,Random Forest,1.811036,1.598093e+15
3,XGB,1.893224,1.649786e+15


##Gradient Boosting

In [83]:
model = GradientBoostingRegressor()

my_pipeline = Pipeline(steps=[('m', model)])

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_train, Y_train, 
                        cv=cv,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'Gradient Boosting',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)


In [84]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15
1,Decision Tree,2.250492,1.362130e+15
2,Random Forest,1.811036,1.598093e+15
3,XGB,1.893224,1.649786e+15
4,Gradient Boosting,1.856124,1.593896e+15


##Histogram-Based Gradient Boosting

In [85]:
model = HistGradientBoostingRegressor()

my_pipeline = Pipeline(steps=[('m', model)])

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_test, Y_test, #test
                        cv=cv,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'HistGradientBoosting',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)


In [86]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15
1,Decision Tree,2.250492,1.362130e+15
2,Random Forest,1.811036,1.598093e+15
3,XGB,1.893224,1.649786e+15
4,Gradient Boosting,1.856124,1.593896e+15
5,HistGradientBoosting,1.777858,1.510443e+15


##LightGBM

In [87]:
# lightgbm for regression

model = LGBMRegressor()

my_pipeline = Pipeline(steps=[('m', model)])

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

#Define as metricas que quer utilizar.
scoring = {'mae' :'neg_mean_absolute_error',
           'mape': 'neg_mean_absolute_percentage_error'}

scores = cross_validate(my_pipeline, 
                        X_test, Y_test, #test
                        cv=cv,
                        scoring=scoring)

# Criar um dicionario com os resultados 
resultado = {'Modelo'     : 'Lightgbm',
             'MAE'        : -1*np.mean(scores['test_mae']),
             'MAPE'       : -1*np.mean(scores['test_mape'])}

Metrics = Metrics.append(resultado,ignore_index=True)


In [88]:
Metrics

,Modelo,MAE,MAPE
0,LinearRegression,1.988747,1.706029e+15
1,Decision Tree,2.250492,1.362130e+15
2,Random Forest,1.811036,1.598093e+15
3,XGB,1.893224,1.649786e+15
4,Gradient Boosting,1.856124,1.593896e+15
5,HistGradientBoosting,1.777858,1.510443e+15
6,Lightgbm,1.767505,1.510316e+15
